In [ ]:
import os
import numpy as np
from functions_InverseSolutions import applyBeamformer, showResult, prepareInverseSolution_group, computeStatistic
import matplotlib as plt
%matplotlib inline

import mne
from mayavi import mlab

#import cmasher as cmr

mlab.init_notebook('png')
mne.viz.set_3d_backend('mayavi')

In [ ]:
sensors = 'mag'
tmin, tmax = -0.1, 0.6

mainFolder = "..\Data\\"
print('main folder folder: ', mainFolder)

meg_MainFolder = mainFolder + "MEG_Data\Data="
print('meg main folder folder: ', meg_MainFolder)

dataFolder = meg_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Data folder: ', dataFolder)

source_MainFolder = "..\SourceLocalization\SourceEstimates\Data="
sourceFolder = source_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Source folder: ', sourceFolder)

subjects_dir = '..\SourceLocalization\subjects\\'
print('Subjects directory: ', subjects_dir)

forwardModelsFolder = '..\SourceLocalization\ForwardModels\\'
print('Forward models folder: ', forwardModelsFolder)

spatialFiltersFolder = '..\SourceLocalization\SpatialFilters\\Data='
spatialFiltersFolder = spatialFiltersFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Spatial filters folder: ', spatialFiltersFolder)

statResultsFolder  = '..\SourceLocalization\Results\\Data='
statResultsFolder = statResultsFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Statisctics results folder: ', statResultsFolder)

classifiers_MainFolder = "..\Classifiers\Data="    
clsfFolder = classifiers_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Classifiers folder: ', clsfFolder)

# Pick the classifiers based on the their training window
peak_indices = [20, 50]
print('Peak indices: ', peak_indices)
    
# Task name for the classifiers
task_name = 'all_predLevel'
print('Task name: ', task_name)

ids_filename = 'Participant_ids.txt'
participants_filename = 'Participants.txt'

# All subjects
s_id_list_all = np.loadtxt(mainFolder+ids_filename, dtype=str)
s_id_list_all = s_id_list_all.tolist()

participant_names = np.loadtxt(mainFolder+participants_filename, dtype=str)
participant_names = participant_names.tolist()

print('Number of subjects: ', len(participant_names))

if os.path.exists(subjects_dir + '\\' + participant_names[0] + '\surf\\'):  
    print('exists!')


In [ ]:
#### Read fsaverage

# Read fsaverage
src_ave = mne.read_source_spaces(subjects_dir + 'fsaverage\\bem\\fsaverage-ico-5-src.fif')
fsave_vertices = [s['vertno'] for s in src_ave]

#### Choose the conditions for contrasting

conditions_8 = ['living_omission_8_nores', 'living_omission_8_corr', 'living_omission_8_incorr',
                 'object_omission_8_nores', 'object_omission_8_corr', 'object_omission_8_incorr']

conditions_10 = ['living_omission_10_nores', 'living_omission_10_corr', 'living_omission_10_incorr',
                 'object_omission_10_nores', 'object_omission_10_corr', 'object_omission_10_incorr']
condition_names =  ["omi"]

n_subjects = len(s_id_list_all)
print('number of subjects: ', n_subjects)

print('conditions 80%: \n', conditions_8)
print('conditions 100%: \n', conditions_10)

### Define time limits for inverse solutions

tminData, tmaxData = 0.1, 0.4  #0.1, 0.4 #-0.04, 0
print('tmin for data: ', tminData)
print('tmax for data: ', tmaxData)
tminNoiseCov, tmaxNoiseCov = -0.1, -0.05
print('tmin for noise covariance: ', tminNoiseCov)
print('tmax for noise covariance: ', tmaxNoiseCov)
tminEpoch = -0.1
print('tmin for generated epochs: ', tminEpoch)
smoothAmount = 70
print('Smoothing amount: ', smoothAmount)
inv_sol_method = 'beamformer'
print('Inverse solution method: ', inv_sol_method)
tstep=0.01

### Use beamformer for computing source estimates

print('======= Applying Beamformer ========')
# 80%
# time range of interest [0.1, 0.4]
stc_omi_8_filename = 'stc_fsave_omi_8_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'
if os.path.exists(statResultsFolder + stc_omi_8_filename) != True:

    stc_fsave_omi_8, n_times, tstep = applyBeamformer(conditions_8, s_id_list_all, n_subjects, participant_names, tminData,
                                                      tmaxData, tminNoiseCov, tmaxNoiseCov, tminEpoch, smoothAmount,
                                                      task_name, clsfFolder, peak_indices, src_ave, spatialFiltersFolder,
                                                      subjects_dir, dataFolder, sensors, forwardModelsFolder)

    print('stc omi is saved in ', stc_omi_8_filename)
    np.save(statResultsFolder + stc_omi_8_filename, stc_fsave_omi_8)

else: # If you have the source estimates values saved already, load them

    # time range of interest [0.1, 0.4]
    tminData_cls_tmp, tmaxData_cls_tmp = 0.1, 0.4
    stc_omi_8_filename = 'stc_fsave_omi_8_onActivationMaps_'+inv_sol_method+'_'+str(tminData_cls_tmp)+'_'+str(tmaxData_cls_tmp)+'_sm='+str(smoothAmount)+'.npy'
    print(statResultsFolder+stc_omi_8_filename)
    stc_fsave_omi_8_clfRange = np.load(statResultsFolder+stc_omi_8_filename)
    print('shape of omi 80% source estimates: ', stc_fsave_omi_8_clfRange.shape)
    print(np.where(stc_fsave_omi_8_clfRange == 0))

    ## Baseline [-0.04, 0]
    tminData_baseline_tmp, tmaxData_baseline_tmp = -0.04, 0
    stc_omi_8_filename_baseline = 'stc_fsave_omi_8_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
    print(stc_omi_8_filename_baseline)
    stc_fsave_omi_8_baseline = np.load(statResultsFolder+stc_omi_8_filename_baseline)
    print('shape of omi 80% baseline source estimates: ', stc_fsave_omi_8_baseline.shape)
    print(np.where(stc_fsave_omi_8_baseline == 0))



#100%
stc_omi_10_filename = 'stc_fsave_omi_10_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'
if os.path.exists(statResultsFolder + stc_omi_10_filename) != True:
    stc_fsave_omi_10, n_times, tstep = applyBeamformer(conditions_10, s_id_list_all, n_subjects, participant_names, tminData,
                                                       tmaxData, tminNoiseCov, tmaxNoiseCov, tminEpoch, smoothAmount,
                                                       task_name, clsfFolder, peak_indices, src_ave, spatialFiltersFolder,
                                                       subjects_dir, dataFolder, sensors, forwardModelsFolder)

    print('stc omi is saved in ', stc_omi_10_filename)
    np.save(statResultsFolder + stc_omi_10_filename, stc_fsave_omi_10)


else:
    # time range of interest [0.1, 0.4]

    tminData_cls_tmp, tmaxData_cls_tmp = 0.1, 0.4
    stc_omi_10_filename = 'stc_fsave_omi_10_onActivationMaps_'+inv_sol_method+'_'+str(tminData_cls_tmp)+'_'+str(tmaxData_cls_tmp)+'_sm='+str(smoothAmount)+'.npy'
    print(statResultsFolder+stc_omi_10_filename)
    stc_fsave_omi_10_clfRange = np.load(statResultsFolder+stc_omi_10_filename)
    print('shape of omi 100% source estimates: ', stc_fsave_omi_10_clfRange.shape)
    print(np.where(stc_fsave_omi_10_clfRange == 0))

    ## Baseline [-0.04, 0]
    tminData_baseline_tmp, tmaxData_baseline_tmp = -0.04, 0
    # 100%
    stc_omi_10_filename_baseline = 'stc_fsave_omi_10_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
    print(stc_omi_10_filename_baseline)
    stc_fsave_omi_10_baseline = np.load(statResultsFolder+stc_omi_10_filename_baseline)
    print('shape of omi 100% baseline source estimates: ', stc_fsave_omi_10_baseline.shape)
    print(np.where(stc_fsave_omi_10_baseline == 0))

### Check data before doing anything!!

#print(stc_fsave_omi_8_baseline)
#print(stc_fsave_omi_10_baseline)
#print(stc_fsave_omi_8_clfRange)
#print(stc_fsave_omi_10_clfRange)
print(stc_fsave_omi_8_clfRange.shape)

# Compute group-level relative change

### 1) Take mean 

# 80%
# take mean over participants (dimension 2)
stc_fsave_omi_8_clfRange_avg = np.mean(stc_fsave_omi_8_clfRange, axis=2)
print('Shape stc_fsave_omi_8_clfRange_avg after avg over subjects: ', stc_fsave_omi_8_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_8_clfRange_avg= np.mean(stc_fsave_omi_8_clfRange_avg, axis=1)
print('Shape stc_fsave_omi_8_clfRange_avg after avg across time: ', stc_fsave_omi_8_clfRange_avg.shape)
print('stc_fsave_omi_8_clfRange_avg shape: ', stc_fsave_omi_8_clfRange_avg.shape)

print('stc_fsave_omi_8_clfRange_avg: ', stc_fsave_omi_8_clfRange_avg)

# take mean over participants (dimension 2)
stc_fsave_omi_8_baseline_avg = np.mean(stc_fsave_omi_8_baseline, axis=2)
print('Shape stc_fsave_omi_8_baseline_avg after avg over subjects: ', stc_fsave_omi_8_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_8_baseline_avg = np.mean(stc_fsave_omi_8_baseline_avg, axis=1)
print('Shape stc_fsave_omi_8_baseline_avg after avg across time: ', stc_fsave_omi_8_baseline_avg.shape)
print('stc_fsave_omi_8_baseline_avg shape: ', stc_fsave_omi_8_baseline_avg.shape)

print('stc_fsave_omi_8_baseline_avg: ', stc_fsave_omi_8_baseline_avg)

# 100%
# take mean over participants (dimension 2)
stc_fsave_omi_10_clfRange_avg = np.mean(stc_fsave_omi_10_clfRange, axis=2)
print('Shape stc_fsave_omi_10_clfRange_avg after avg over subjects: ', stc_fsave_omi_10_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_10_clfRange_avg= np.mean(stc_fsave_omi_10_clfRange_avg, axis=1)
print('Shape stc_fsave_omi_10_clfRange_avg after avg across time: ', stc_fsave_omi_10_clfRange_avg.shape)
print('stc_fsave_omi_10_clfRange_avg shape: ', stc_fsave_omi_10_clfRange_avg.shape)

print('stc_fsave_omi_10_clfRange_avg: ', stc_fsave_omi_10_clfRange_avg)

# take mean over participants (dimension 2)
stc_fsave_omi_10_baseline_avg = np.mean(stc_fsave_omi_10_baseline, axis=2)
print('Shape stc_fsave_omi_10_baseline_avg after avg over subjects: ', stc_fsave_omi_10_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_10_baseline_avg = np.mean(stc_fsave_omi_10_baseline_avg, axis=1)
print('Shape stc_fsave_omi_10_baseline_avg after avg across time: ', stc_fsave_omi_10_baseline_avg.shape)
print('stc_fsave_omi_10_baseline_avg shape: ', stc_fsave_omi_10_baseline_avg.shape)

print('stc_fsave_omi_10_baseline_avg: ', stc_fsave_omi_10_baseline_avg)

In [ ]:
### 2) Generate stc group

#80%
stc_cls_8 = prepareInverseSolution_group(stc_fsave_omi_8_clfRange_avg, subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_baseline_8 = prepareInverseSolution_group(stc_fsave_omi_8_baseline_avg, subjects_dir, tstep=tstep,
                                            tmin_tmp=tminData_baseline_tmp)

#100%
stc_cls_10 = prepareInverseSolution_group(stc_fsave_omi_10_clfRange_avg, subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_baseline_10 = prepareInverseSolution_group(stc_fsave_omi_10_baseline_avg, subjects_dir, tstep=tstep,
                                            tmin_tmp=tminData_baseline_tmp)

### Plot
# 80% cls
showResult('fsaverage', sourceFolder, stc_cls_8, 'omi_8_clf_'+inv_sol_method+'_sm='+str(smoothAmount), subjects_dir,
           minimum=1300000, maximum=3000000)
# 80% baseline
showResult('fsaverage', sourceFolder, stc_baseline_8, 'omi_8_baseline_'+inv_sol_method+'_sm='+str(smoothAmount),
           subjects_dir, minimum=99000, maximum=440000)
# 100% cls
showResult('fsaverage', sourceFolder, stc_cls_10, 'omi_10_clf_'+inv_sol_method+'_sm='+str(smoothAmount), subjects_dir,
           minimum=1300000, maximum=3000000)
# 100% baseline
showResult('fsaverage', sourceFolder, stc_baseline_10, 'omi_10_baseline_'+inv_sol_method+'_sm='+str(smoothAmount),
           subjects_dir, minimum=99000, maximum=440000)

### 3) Average relative change

#80% avg
stc_fsave_omi_8_diff_avg = 100*(stc_fsave_omi_8_clfRange_avg-stc_fsave_omi_8_baseline_avg)/stc_fsave_omi_8_baseline_avg

# 100% avg
stc_fsave_omi_10_diff_avg = 100*(stc_fsave_omi_10_clfRange_avg-stc_fsave_omi_10_baseline_avg)/stc_fsave_omi_10_baseline_avg

print('Minimum of 80%: ', min(stc_fsave_omi_8_diff_avg))
print('Maximum of 80%: ', max(stc_fsave_omi_8_diff_avg))

print('Minimum of 100%: ', min(stc_fsave_omi_10_diff_avg))
print('Maximum of 100%: ', max(stc_fsave_omi_10_diff_avg))

thresh_8 = 770 #160
thresh_10 = 770
print(np.where(stc_fsave_omi_8_diff_avg >thresh_8)[0].shape)
print(np.where(stc_fsave_omi_10_diff_avg >thresh_10)[0].shape)

## zero out the values below threshold
# 80%
for i in range(stc_fsave_omi_8_diff_avg.shape[0]):
    if stc_fsave_omi_8_diff_avg[i] < thresh_8:
        stc_fsave_omi_8_diff_avg[i] = 0
        
# 100%
for i in range(stc_fsave_omi_10_diff_avg.shape[0]):
    if stc_fsave_omi_10_diff_avg[i] < thresh_10:
        stc_fsave_omi_10_diff_avg[i] = 0

stc_diff_8 = prepareInverseSolution_group(stc_fsave_omi_8_diff_avg, subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_diff_10 = prepareInverseSolution_group(stc_fsave_omi_10_diff_avg, subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)

#### PLOT
#80%
print('Minimum of 80%: ', min(stc_diff_8.data))
print('Maximum of 80%: ', max(stc_diff_8.data))
showResult('fsaverage', sourceFolder, stc_diff_8, 
           '2omi_8_change_'+inv_sol_method+'_sm='+str(smoothAmount) +'_pthresh=' + str(thresh_8), subjects_dir,
           minimum=600, maximum=1100)

#100%
print('Minimum of 100%: ', min(stc_diff_10.data))
print('Maximum of 100%: ', max(stc_diff_10.data))
showResult('fsaverage', sourceFolder, stc_diff_10, 
           '2omi_10_change_'+inv_sol_method+'_sm='+str(smoothAmount) +'_pthresh=' + str(thresh_10), subjects_dir,
           minimum=600, maximum=1100)

# Compute relative change for each participant at 80 and 100 level

In [ ]:
### 1) Compute relative change

#80%
# collapse time range by averaging over time points -> output will have the size of n_vertices x nsubjects
stc_fsave_omi_8_clfRange_avgOverTime = np.mean(stc_fsave_omi_8_clfRange, axis=1)
stc_fsave_omi_8_baseline_avgOverTime = np.mean(stc_fsave_omi_8_baseline, axis=1)

stc_fsave_omi_8_diff = np.zeros(stc_fsave_omi_8_clfRange_avgOverTime.shape)
print('shape of stc_fsave_omi_8_diff: ', stc_fsave_omi_8_diff.shape)
for j in range(stc_fsave_omi_8_diff.shape[1]):
    stc_fsave_omi_8_diff[:,j] = 100*(stc_fsave_omi_8_clfRange_avgOverTime[:,j]-stc_fsave_omi_8_baseline_avgOverTime[:,j])/stc_fsave_omi_8_baseline_avgOverTime[:,j]

print('stc_fsave_omi_8_diff: ', stc_fsave_omi_8_diff.shape)

# 100%
# collapse time range by averaging over time points -> output will have the size of n_vertices x nsubjects
stc_fsave_omi_10_clfRange_avgOverTime = np.mean(stc_fsave_omi_10_clfRange, axis=1)
stc_fsave_omi_10_baseline_avgOverTime = np.mean(stc_fsave_omi_10_baseline, axis=1)

stc_fsave_omi_10_diff = np.zeros(stc_fsave_omi_10_clfRange_avgOverTime.shape)

for j in range(stc_fsave_omi_10_diff.shape[1]):
    stc_fsave_omi_10_diff[:,j] = 100*(stc_fsave_omi_10_clfRange_avgOverTime[:,j]-stc_fsave_omi_10_baseline_avgOverTime[:,j])/stc_fsave_omi_10_baseline_avgOverTime[:,j]

print('stc_fsave_omi_10_diff: ', stc_fsave_omi_10_diff)
print(stc_fsave_omi_8_diff[0,0])
print(stc_fsave_omi_10_diff[0,0])

for i in range(stc_fsave_omi_10_diff.shape[1]):
    print('i: ', i)
    print('N=', len(np.where(stc_fsave_omi_8_diff[:,i]-stc_fsave_omi_10_diff[:,i]>0)[0]))

### 2) Stats to contrast 80% predictability and 100% predictability
pthresh = 0.05
print('computing the stats..')

#omissions
fvalues_o, pvalues_o = computeStatistic(stc_fsave_omi_10_diff, stc_fsave_omi_8_diff)

# save values for omissions
fvalue_filename_o = statResultsFolder+'fvalues_all_omi_activations_8vs10_'+inv_sol_method+'_sm='+str(smoothAmount)+'.npy'
print(fvalue_filename_o)
pvalue_filename_o = statResultsFolder+'pvalues_all_omi_activations_8vs10_'+inv_sol_method+'_sm='+str(smoothAmount)+'.npy'
print('saving stats...')
np.save(fvalue_filename_o, fvalues_o)
np.save(pvalue_filename_o, pvalues_o)

print('Significant p vals omi: ', np.where(pvalues_o <= pthresh))
fvalues_o_new = np.copy(fvalues_o)
pvalues_o_new = np.copy(pvalues_o)

for j in range(len(fvalues_o)):
    #omissions
    if pvalues_o[j] > pthresh:
        pvalues_o_new[j] = 1
        fvalues_o_new[j] = 0

#omissions
rejets_o, pvalues_o_corrected = mne.stats.fdr_correction(pvalues_o)
print('omissions-corrrected:')
print(np.where(pvalues_o_corrected<= pthresh))

print('generate stc for stats..')
# omissions
stc_f_omi = prepareInverseSolution_group(fvalues_o_new, subjects_dir, tstep=tstep)
#stc_f_omi = prepareInverseSolution_group(1-pvalues_o_new, subjects_dir, tstep=tstep)

#omissions
#lateral
v_mi, v_ma = 0, 2.09  #-1.5, 1.5
showResult('fsaverage', sourceFolder, stc_f_omi, 
           'pval_predLevel_omi_activations_8vs10_pthresholded_'+inv_sol_method+'_sm='+str(smoothAmount)+'_pthresh='+str(pthresh),
           minimum=v_mi, maximum=v_ma, cmap='coolwarm', sequentialCmap=False) #twilight_shifted


#del fvalues_o, pvalues_o, pvalues_o_corrected, fvalues_o_new, stc_f_omi

print('fvalues_o_new min: ', min(abs(fvalues_o_new[fvalues_o_new != 0])))
print('fvalues_o_new max: ', max(abs(fvalues_o_new[fvalues_o_new != 0])))


### 3) Compute the difference between 80% and 100%
cmap = plt.get_cmap('cmr.seasons')
mask = 300

stc_fsave_omi_diffBetween_10_8 = np.zeros((stc_fsave_omi_10_diff.shape[0], stc_fsave_omi_10_diff.shape[1]),)
for i in range(stc_fsave_omi_10_diff.shape[0]):
    for j in range(stc_fsave_omi_10_diff.shape[1]):
        stc_fsave_omi_diffBetween_10_8[i,j] = stc_fsave_omi_10_diff[i,j]-stc_fsave_omi_8_diff[i,j]

stc_fsave_omi_diffBetween_10_8_avg = np.mean(stc_fsave_omi_diffBetween_10_8, axis=1)
stc_fsave_omi_diffBetween_10_8_avg_masked = stc_fsave_omi_diffBetween_10_8_avg.copy()

for i in range(stc_fsave_omi_diffBetween_10_8_avg_masked.shape[0]):
    if stc_fsave_omi_diffBetween_10_8_avg_masked[i] > 0:
        stc_fsave_omi_diffBetween_10_8_avg_masked[i] = 1
    else:
        stc_fsave_omi_diffBetween_10_8_avg_masked[i] = -1


print('Min of stc_fsave_omi_diffBetween_10_8_avg: ', min(stc_fsave_omi_diffBetween_10_8_avg))
print('Max of stc_fsave_omi_diffBetween_10_8_avg: ', max(stc_fsave_omi_diffBetween_10_8_avg))

stc_fsave_omi_diffBetween_10_8_avg_new = stc_fsave_omi_diffBetween_10_8_avg.copy()
for s in range(stc_fsave_omi_diffBetween_10_8_avg_new.shape[0]):
    if stc_fsave_omi_diffBetween_10_8_avg[s]  < mask and stc_fsave_omi_diffBetween_10_8_avg[s] > -mask:
        stc_fsave_omi_diffBetween_10_8_avg_new[s] = 0
        

print(len(np.where(stc_fsave_omi_diffBetween_10_8_avg_new < 1)[0]))
stc_fsave_omi_diffBetween_10_8_avg_source = prepareInverseSolution_group(stc_fsave_omi_diffBetween_10_8_avg_new,
                                                                         subjects_dir, tstep=tstep, tmin_tmp=tminData_cls_tmp)


showResult('fsaverage', sourceFolder, stc_fsave_omi_diffBetween_10_8_avg_source, 
           'omi_diffBetween_10_8_'+inv_sol_method+'_sm='+str(smoothAmount) + 'mask=' + str(mask), 
           minimum=0, mid=500, maximum=1500, cmap='seismic', sequentialCmap=False)


print(stc_fsave_omi_diffBetween_10_8_avg_new[stc_fsave_omi_diffBetween_10_8_avg_new >= 0].shape)
print(stc_fsave_omi_diffBetween_10_8_avg_new[stc_fsave_omi_diffBetween_10_8_avg_new < 0].shape)
print(stc_fsave_omi_10_diff.shape)